# Text Analysis

In this lab we'll do a short text analysis so that you start to become familiar with the packages and tools available to you in Python to work with text data. Nothing here will be very in-depth - it's supposed to be able to be completed in a short period of time after all. But, it will give you a starting point for your final assignment and projects, should you want to analyze text data.

## Data Science Question
In this short project, we're going to answer the question: *For each presidential inauguration, which word is most unique?* 

To do this, we'll use the text from each Inaugural address in American history and carry out a TF-IDF analaysis.

Secondarily, we'll think about whether these words make sense in the context of the history at the time and visualize words uniqueness over the course of history.

# Part I : Setup & Data Wrangling

This lab uses a number of different functions across multiple packages. **Run the following code cell and take a look at each package we'll be using below. Make sure you understand what the package is used for. Be sure to familiarize yourself with anything that you're not yet familiar with.**

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

# Import nltk package 
# NLTK provides support for a wide variety of text processing tasks: 
# tokenization, stemming, proper name identification, part of speech identification, etc. 
#   PennTreeBank word tokenizer 
#   English language stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# scikit-learn imports
#   TF-IDF Vectorizer that first removes widely used words in the dataset and then transforms test data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# import re for regular expression
import re

## seaborn for plotting
import seaborn as sns
sns.set(font_scale=1.2, style="white")

# import matplotlib for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
# set plotting size parameter
plt.rcParams['figure.figsize'] = (12, 5)

# improve resolution
%config InlineBackend.figure_format ='retina'

To get started on your text analysis using the `nltk` package, run the code below to **download the NLTK English tokenizer ('punkt'), stopwords of all languages ('stopwords') from `nltk`, and the inaugural dataset from `nltk` ('inaugural')**. To determine what code you'll need to do this, you can explore the `download` method [here](https://www.nltk.org/) or their book [here](http://www.nltk.org/book/).

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('inaugural')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rabwa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rabwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package inaugural to
[nltk_data]     C:\Users\rabwa\AppData\Roaming\nltk_data...
[nltk_data]   Package inaugural is already up-to-date!


True

Now that you have downloaded a few of the datasets you'll need, **import the `inaugural` dataset from `nltk.corpus`.**

In [3]:
# YOUR CODE HERE
from nltk.corpus import inaugural

In [4]:
assert inaugural

If all is working well, the following cell should display the files included in this dataset. 

In [5]:
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

As you can see there is one file from each address. And, you'll note that the filename includes the year of each address. We'll want to use that address later, so **write code that extracts each year from the filename and stores it as a list. Assign this list to the variable `years`.**  

In [6]:
# YOUR CODE HERE
years = [fileid[:4] for fileid in inaugural.fileids()]

In [7]:
assert len(years) == len(inaugural.fileids())
assert years[1] == '1793'

Let's take a look at one of these addresses. We'll pick a short one - Washington's *second* address. **Run the code below to take a look.**

In [8]:
# see Washington's Second Inaugural Address
inaugural.raw('1793-Washington.txt')

'Fellow citizens, I am again called upon by the voice of my country to execute the functions of its Chief Magistrate. When the occasion proper for it shall arrive, I shall endeavor to express the high sense I entertain of this distinguished honor, and of the confidence which has been reposed in me by the people of united America.\n\nPrevious to the execution of any official act of the President the Constitution requires an oath of office. This oath I am now about to take, and in your presence: That if it shall be found during my administration of the Government I have in any instance violated willingly or knowingly the injunctions thereof, I may (besides incurring constitutional punishment) be subject to the upbraidings of all who are now witnesses of the present solemn ceremony.\n\n \n'

You'll notice that there are some new line characters, as well as a colon, some commas, some periods. We're really only interested in the words though for TF-IDF, so let's remove all punctuation. **Write code that returns a list (`text`), where each element in the list includes the text as above, but with:
- punctuation removed 
- each word separated by a space
- all words are lower case (i.e. "Constitution" should be "constitution)

Assign this to the variable `text`.

In [9]:
# YOUR CODE HERE
text = [re.sub(r'[^A-Za-z0-9]+', ' ', x) for x in [inaugural.raw(file_id) for file_id in inaugural.fileids()]]
text = list(map(str.lower, text))

In [10]:
assert isinstance(text, list)
assert len(text) == 59
out = re.search('^fellow',text[0])
assert out != None

AssertionError: 

If you've done this correctly and you **run the following cell, all punctuation should be stripped from the text, so that you only see the words from Washington's second address, separated by spaces, with all words lowercase.**

In [11]:
text[1]

'fellow citizens i am again called upon by the voice of my country to execute the functions of its chief magistrate when the occasion proper for it shall arrive i shall endeavor to express the high sense i entertain of this distinguished honor and of the confidence which has been reposed in me by the people of united america previous to the execution of any official act of the president the constitution requires an oath of office this oath i am now about to take and in your presence that if it shall be found during my administration of the government i have in any instance violated willingly or knowingly the injunctions thereof i may besides incurring constitutional punishment be subject to the upbraidings of all who are now witnesses of the present solemn ceremony '

With that, you now have a dataset ready for analysis by TF-IDF!

# Part II : Text Analysis

To get started on your TF-IDF analysis, you'll first want to **create a `TfidfVectorizer` object to transform your text data into vectors. Assign this `TfidfVectorizer` object to `tfidf`.**

In this object, you'll need to **pass five arguments to initialize `tfidf`**: 
* set to apply TF scaling: `sublinear_tf=True`
* analyze at the word-level: `analyzer='word'`
* set maximum number of unique words: ` max_features=2000`
* specify that you want to tokenize the data using the word_tokenizer from NLTK: `tokenizer=word_tokenize`
* remove English language stop words: `stop_words=stopwords.words("english")`

In [12]:
# YOUR CODE HERE
stop_words_processed = set(word_tokenize(' '.join(stopwords.words('english'))))
tfidf = TfidfVectorizer(sublinear_tf=True,
                        analyzer='word',
                        max_features=2000,
                        tokenizer=word_tokenize,
                        stop_words=stop_words_processed)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\rabwa/nltk_data'
    - 'C:\\Users\\rabwa\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'C:\\Users\\rabwa\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'C:\\Users\\rabwa\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\rabwa\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [13]:
assert tfidf.analyzer == 'word'
assert tfidf.max_features == 2000
assert tfidf.tokenizer == word_tokenize

NameError: name 'tfidf' is not defined

Now, it's time to calculate TF-IDF for words across our corpus of Inaugural addresses! 

To do this:

1. generate a DataFrame `inaug_tfidf` using the `tfidf.fit_transform` function to calculate TF-IDF on your `text` variable. 
2. Be sure that your index here is the year of the address and the columns are named with the columns of the words the values represent. The `get_feature_names` method from `tfidf` may help you accomplish the columns name assignment. And the `years` you created earlier may help you with the indices.

In [14]:
# YOUR CODE HERE
texts = [inaugural.raw(file_id) for file_id in inaugural.fileids()]
cleaned_texts = [re.sub(r'[^\w\s]', '', text).lower() for text in texts]
tfidf_matrix = tfidf.fit_transform(cleaned_texts)
inaug_tfidf = pd.DataFrame(tfidf_matrix.toarray(), index=years, columns=tfidf.get_feature_names())

NameError: name 'tfidf' is not defined

In [15]:
assert len(inaug_tfidf.index) == len(years)
assert len(inaug_tfidf.columns) == 2000
assert inaug_tfidf.shape == (59, 2000)

NameError: name 'inaug_tfidf' is not defined

# Part 3: Results

We're almost there. We now have a DataFrame that includes the TF-IDF for the top 2000 words in our corpus! **Now, you'll want to extract the single most unique word from each address. Assign this information (most likely a Series object) to the variable `most_unique`.**

In [16]:
# YOUR CODE HERE
# Extract the most unique word from each address
most_unique = inaug_tfidf.idxmax(axis = 1)

most_unique

NameError: name 'inaug_tfidf' is not defined

In [17]:
assert len(most_unique) == 59
assert most_unique[0] == 'article'
assert most_unique[-1] == 'story'

NameError: name 'most_unique' is not defined

Take a look through this list of most unique words over time. Do they make sense based on what you know about American history? Do any surprise you?

With that part of our Analysis done, one thing that stuck out to me in this list is the fact that "british" was the most unique word to the 1813 inaugural address. This made sense to me - it was early in American history and we had only recently left British rule. But, I was curious to see whether or not 'british' would show up uniquely (albeit less uniquely) in any later addresses. **Generate a line plot that plots the TF-IDF for the word "british" on the y-axis. Plot year on the x-axis.**

In [18]:
# YOUR CODE HERE

f1 = plt.gcf()

<Figure size 1200x500 with 0 Axes>

In [19]:
assert f1.gca().has_data()

AssertionError: 

Here you should see that over time "british" peaked in inaugural addresses at a few interesting points throughout history. What about some other words?

Using a similar approach, **plot TF-IDF for "british", "america", "war", and "jobs". Take a look at the trends over time. Feel free to look at other words' trends over time.**

In [20]:
# YOUR CODE HERE
raise NotImplementedError()

f2 = plt.gcf()

NotImplementedError: 

In [21]:
assert f2.gca().has_data()

NameError: name 'f2' is not defined

You should see that the mention of "america" happened frequently in the country's infancy, but then became less common, whereas "british was really common early on and "jobs" has really only become applicable in recent innaugural addresses.

As with all analysis, TF-IDF is not without its limitations. Let's take a look at how our results change if we change the `max_features` result in our analysis above to include 4000 words (rather than 2000). **Redo the analysis to 1) calculate TF-IDF for these 4000 words, 2) identify the word with the highest TF-IDF in each year (assignt his to `most_unique_4000`, and 3) generate a dataframe with the most common word from each analysis.Then, take a look to see how changing one argument in your analysis can affect your results! Finally, you can regenerate line plots if you're interseted to see how your plots have changed in this new analysis.**

Define a tfidfvectorizer object as `tfidf2`

In [22]:
# define tfidfvectorizer object as tfidf2
tfidf2 = None
# YOUR CODE HERE
raise NotImplementedError()


NotImplementedError: 

In [23]:
assert tfidf2.max_features == 4000

AttributeError: 'NoneType' object has no attribute 'max_features'

In [24]:
# calculate TF-DF on input text
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [25]:
assert list(inaug_tfidf2.index) == years
assert inaug_tfidf2.shape == (59,4000)

NameError: name 'inaug_tfidf2' is not defined

In [26]:
# identify most uniuqe word each year from new model
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [27]:
assert len(most_unique_4000) == 59
assert most_unique_4000[0] == 'immutable'
assert most_unique_4000[-1] == 'virus'

NameError: name 'most_unique_4000' is not defined

### Compare the Results
Join `most_unique` from the original model with this new list `most_unique_4000`
in a single dataframe to compare word each year. Name this dataframe `df_unique`

In [28]:
df_unique = None
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [29]:
assert df_unique.shape == (59,2)

for i in range(df_unique.shape[0]):
    assert (df_unique[0] == most_unique)[i] or (df_unique[1] == most_unique)[i]

AttributeError: 'NoneType' object has no attribute 'shape'

In [30]:
# regenerate plot
# YOUR CODE HERE
raise NotImplementedError()

f3 = plt.gcf()

NotImplementedError: 

In [31]:
assert f3.gca().has_data()

NameError: name 'f3' is not defined

Good work getting comfortable working with text data here...and hopefully learning a bit more about Inaugural Addresses over time. Go ahead and submit your discussion lab!